In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr 22 11:57:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# import data
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from datasets import Dataset, DatasetDict
import evaluate


In [8]:
classifier = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=77)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [27]:
# Reading Training Data - contains banking 77 dataset
train_data_readin = pd.read_csv('train.csv')
train_data_readin = train_data_readin.loc[:,["clean_text", "label"]]

In [29]:
# Create a label mapping dictionary for the 77 classes
# First get all unique labels
unique_labels = sorted(train_data_readin["label"].unique())
# Create mapping dictionary: string label -> integer
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

# Convert string labels to integers
train_data_readin["label_id"] = train_data_readin["label"].map(label_to_id)

In [31]:
train_data_readin.head(5)

,clean_text,label,label_id
0,I am still waiting on my card?,card_arrival,12
1,What can I do if my card still hasn't arrived ...,card_arrival,12
2,I have been waiting over a week. Is the card s...,card_arrival,12
3,Can I track my card while it is in the process...,card_arrival,12
4,"How do I know if I will get my card, or if it ...",card_arrival,12


In [32]:
# Split into train and validation sets
train_data, val_data = train_test_split(train_data_readin, train_size=0.8,
                                        stratify=train_data_readin.label_id,
                                        random_state=1234)

In [33]:
print("training data : ",train_data.shape)
print("validation data : ",val_data.shape)

training data :  (8002, 3)
validation data :  (2001, 3)


In [34]:
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "eval": Dataset.from_pandas(val_data)})

In [14]:
print("Dataset Dict:\n", raw_datasets)
print("\n\nTrain's features:\n", raw_datasets["train"].features)
print("\n\nFirst row of Train:\n", raw_datasets["train"][0])

Dataset Dict:
 DatasetDict({
    train: Dataset({
        features: ['clean_text', 'label', '__index_level_0__'],
        num_rows: 8002
    })
    eval: Dataset({
        features: ['clean_text', 'label', '__index_level_0__'],
        num_rows: 2001
    })
})


Train's features:
 {'clean_text': Value(dtype='string', id=None), 'label': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}


First row of Train:
 {'clean_text': "why hasn't my virtual card come yet?", 'label': 'getting_virtual_card', '__index_level_0__': 3090}


In [35]:
# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['clean_text'], truncation=True, padding='max_length', max_length=512)

# Tokenize datasets
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/8002 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'label', 'label_id', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 8002
    })
    eval: Dataset({
        features: ['clean_text', 'label', 'label_id', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 2001
    })
})


In [36]:
#validating for completeness
print(tokenized_datasets["train"][0])

{'clean_text': "why hasn't my virtual card come yet?", 'label': 'getting_virtual_card', 'label_id': 41, '__index_level_0__': 3090, 'input_ids': [101, 2339, 8440, 1005, 1056, 2026, 7484, 4003, 2272, 2664, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [37]:
# Remove unnecessary columns and rename label column
tokenized_datasets = tokenized_datasets.remove_columns(["clean_text", "label", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("label_id", "labels")

In [38]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [39]:
# Define metrics computation function for multiclass
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    accuracy = evaluate.load("accuracy")
    precision = evaluate.load("precision")
    recall = evaluate.load("recall")
    f1 = evaluate.load("f1")

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "precision_macro": precision.compute(predictions=predictions, references=labels, average='macro')["precision"],
        "precision_weighted": precision.compute(predictions=predictions, references=labels, average='weighted')["precision"],
        "recall_macro": recall.compute(predictions=predictions, references=labels, average='macro')["recall"],
        "recall_weighted": recall.compute(predictions=predictions, references=labels, average='weighted')["recall"],
        "f1_macro": f1.compute(predictions=predictions, references=labels, average='macro')["f1"],
        "f1_weighted": f1.compute(predictions=predictions, references=labels, average='weighted')["f1"],
    }

In [40]:
# Define training arguments with class imbalance handling
training_args = TrainingArguments(
    output_dir="distilbert-multiclass-classifier",
    num_train_epochs=5,  # Increased epochs for more complex task
    per_device_train_batch_size=16,  # DistilBERT is smaller than BERT-medium
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",  # Changed to weighted F1 for multiclass
    weight_decay=1e-3,  # Increased for better regularization
    learning_rate=3e-5,
    report_to="none",
    # Add class weights calculation if needed
    # For severe imbalance, you might want to implement custom loss weighting
)

# Initialize trainer
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-40-d5a69874b202>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Precision Weighted,Recall Macro,Recall Weighted,F1 Macro,F1 Weighted
1,3.017800,1.563506,0.741129,0.736100,0.760249,0.708706,0.741129,0.692915,0.720756
2,1.046500,0.641411,0.870565,0.870757,0.879762,0.852022,0.870565,0.846909,0.865109
3,0.450400,0.430118,0.901049,0.907733,0.906622,0.897051,0.901049,0.898624,0.900625
4,0.251100,0.373619,0.906047,0.913629,0.912881,0.903015,0.906047,0.904647,0.906239
5,0.166800,0.355798,0.909045,0.915783,0.914502,0.904856,0.909045,0.907156,0.909139


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TrainOutput(global_step=2505, training_loss=0.9850819076606614, metrics={'train_runtime': 462.7795, 'train_samples_per_second': 86.456, 'train_steps_per_second': 5.413, 'total_flos': 5307109496002560.0, 'train_loss': 0.9850819076606614, 'epoch': 5.0})

In [42]:
# Make predictions on evaluation dataset
eval_predictions = trainer.predict(tokenized_datasets["eval"])
y_pred = np.argmax(eval_predictions.predictions, axis=-1)

# Get the true labels
y_true = np.array(tokenized_datasets["eval"]["labels"])

# Print the classification report for validation data
print("\nValidation Results:")
print(classification_report(y_true, y_pred, digits=3, target_names=[id_to_label[i] for i in range(len(id_to_label))]))


Validation Results:
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up      1.000     0.906     0.951        32
                                activate_my_card      1.000     0.938     0.968        32
                                       age_limit      1.000     1.000     1.000        22
                         apple_pay_or_google_pay      0.962     1.000     0.980        25
                                     atm_support      1.000     0.941     0.970        17
                                automatic_top_up      0.958     0.920     0.939        25
         balance_not_updated_after_bank_transfer      0.867     0.765     0.812        34
balance_not_updated_after_cheque_or_cash_deposit      0.919     0.944     0.932        36
                         beneficiary_not_allowed      1.000     0.903     0.949        31
                                 cancel_transfer      0.969     1.000     0.98

In [43]:
trainer.save_model('/content/drive/MyDrive/Banking-DistilBERT')

# **Test Data Performance Checks**

In [44]:
uploaded = files.upload()

Saving test.csv to test.csv


In [45]:
# Reading Test Data
test_data_readin = pd.read_csv('test.csv')
test_data_readin = test_data_readin.loc[:,["clean_text", "label"]]

# Convert test labels to integers using the same mapping
test_data_readin["label_id"] = test_data_readin["label"].map(label_to_id)

print(test_data_readin.shape)
test_data_readin.head(5)

(3080, 3)


,clean_text,label,label_id
0,How do I locate my card?,card_arrival,12
1,"I still have not received my new card, I order...",card_arrival,12
2,I ordered a card but it has not arrived. Help ...,card_arrival,12
3,Is there a way to know when my card will arrive?,card_arrival,12
4,My card has not arrived yet.,card_arrival,12


In [47]:
# Prepare test dataset
test_df = Dataset.from_pandas(test_data_readin)
test_df = test_df.map(tokenize_function, batched=True)

# Remove only columns that exist in the test dataset
columns_to_remove = ["clean_text", "label"]
existing_columns = [col for col in columns_to_remove if col in test_df.column_names]
test_df = test_df.remove_columns(existing_columns)
test_df = test_df.rename_column("label_id", "labels")

# Make predictions on test data
test_predictions = trainer.predict(test_df)
test_preds = np.argmax(test_predictions.predictions, axis=-1)

# Get true test labels
test_true = np.array(test_df["labels"])

Map:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [48]:
# Get true test labels
test_true = np.array(test_df["labels"])

# Get classification report as a dictionary
target_names = [id_to_label[i] for i in range(len(id_to_label))]
report = classification_report(test_true, test_preds, digits=3,
                              target_names=target_names,
                              output_dict=True)

# Convert the classification report to a DataFrame
report_df = pd.DataFrame(report).transpose()

In [49]:
# Save the classification report to Google Drive
report_path = '/content/drive/My Drive/banking_distilbert_classification_report.csv'
report_df.to_csv(report_path)
print(f"Classification report saved to {report_path}")

Classification report saved to /content/drive/My Drive/banking_distilbert_classification_report.csv


In [51]:
# Print classification report for test data
print("\nTest Results:")
print(classification_report(test_true, test_preds, digits=3, target_names=[id_to_label[i] for i in range(len(id_to_label))]))


Test Results:
                                                  precision    recall  f1-score   support

                           Refund_not_showing_up      1.000     0.950     0.974        40
                                activate_my_card      1.000     0.950     0.974        40
                                       age_limit      0.976     1.000     0.988        40
                         apple_pay_or_google_pay      1.000     1.000     1.000        40
                                     atm_support      1.000     0.950     0.974        40
                                automatic_top_up      0.973     0.900     0.935        40
         balance_not_updated_after_bank_transfer      0.795     0.775     0.785        40
balance_not_updated_after_cheque_or_cash_deposit      0.975     0.975     0.975        40
                         beneficiary_not_allowed      0.921     0.875     0.897        40
                                 cancel_transfer      0.974     0.950     0.962     

In [52]:
# Create a DataFrame with original labels and predictions
predictions_df = pd.DataFrame({
    'true_label_id': test_true,
    'predicted_label_id': test_preds,
    'true_label': [id_to_label[id] for id in test_true],
    'predicted_label': [id_to_label[id] for id in test_preds]
})

# Save detailed predictions to Google Drive
predictions_path = '/content/drive/My Drive/banking_distilbert_predictions.csv'
predictions_df.to_csv(predictions_path, index=False)

In [53]:
predictions_df.head(5)

,true_label_id,predicted_label_id,true_label,predicted_label
0,12,42,card_arrival,lost_or_stolen_card
1,12,12,card_arrival,card_arrival
2,12,12,card_arrival,card_arrival
3,12,13,card_arrival,card_delivery_estimate
4,12,12,card_arrival,card_arrival


In [ ]:
!mkdir -p "/content/drive/MyDrive/fine-tune-envs/"

In [ ]:
!pip freeze > "/content/drive/MyDrive/fine-tune-envs/requirements.txt"